# Advanced Ray Tutorial - Exercise Solutions

First, import everything we'll need and start Ray:

In [1]:
import ray, time, sys
import numpy as np
sys.path.append("../..")
from util.printing import pd, pnd  # convenience methods for printing results.

In [2]:
!../../tools/start-ray.sh


Ray already running or successfully started


In [3]:
ray.init(address='auto', ignore_reinit_error=True)

2020-05-24 14:32:44,456	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:55312',
 'object_store_address': '/tmp/ray/session_2020-05-24_11-24-33_071372_73030/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-24_11-24-33_071372_73030/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-05-24_11-24-33_071372_73030'}

## Exercise 1 in 01: Ray Tasks Revisited

You were asked to convert the regular Python code to Ray code. Here are the three cells appropriately modified.

First, we need the appropriate imports and `ray.init()`.

In [5]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

In [6]:
start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = ray.get(ids)
duration = time.time() - start

In [7]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 

## Exercise 2 in 01: Ray Tasks Revisited

You were asked to use `ray.wait()` with a shorter timeout, `2.5` seconds. First we need to redefine in this notebook the remote functions we used in that lesson:

In [8]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

In [9]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=2.5)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.017)
0: []
10: [ 4.32737218  1.84041995 -3.65869664 -1.18314247  1.17692894 -6.50627178
 -2.56290478 -0.65746464  2.29855075  1.44218294]
Returned   1 completed tasks. (elapsed time:  5.530)
20: [ 1.12572964 -0.1775737  -2.65607606 -2.03323559 -0.90437758  0.25937131
 -2.73127879  0.62081916 -2.23769877  0.41473947  2.8628821  -0.29769767
  1.60093991 -2.80766646 -0.24939007  1.91890681 -1.51658634 -1.7927651
 -0.48840102 -0.07738982]
Returned   2 completed tasks. (elapsed time:  8.535)
30: [-0.52096067 -0.97949442 -0.76375359 -0.12431216 -0.35839995  0.54821904
 -0.36848895  1.32915568 -2.01199297  1.73866514  1.06718706 -4.11578144
  0.46970735 -0.16417912  3.99567667  0.51468331 -0.02258815  3.13481553
  5.07946479 -2.68242857  0.30758541 -0.68127055  0.84779553 -0.7720236
 -2.59236206 -2.15657653  1.76079816  0.92234692  0.76478132 -1.30454804]
40: [ 3.39830416  0.21360206 -2.50844728 -2.46589293 -2.79462406 -2.17230112
  0.06005292 -

For a timeout of `2.5` seconds, the second call to `ray.wait()` times out before two tasks finish, so it only returns one completed task. Why did the third and last iteration not time out? (That is, they both successfully returned two items.) It's because all the tasks were running in parallel so they had time to finish. If you use a shorter timeout, you'll see more time outs, where zero or one items are returned. 

Try `1.5` seconds, where all but one iteration times out and returns one item. The first iteration returns two items.
Try `0.5` seconds, where you'll get several iterations that time out and return zero items, while all the other iterations time out and return one item.

## Exercise 3 in 01: Ray Tasks Revisited

You were asked to convert the code to use Ray, especially `ray.wait()`.

In [10]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = []
waiting_ids = ids
while len(waiting_ids) > 0:
    finished_ids, waiting_ids = ray.wait(waiting_ids)  # We just assign the second list to waiting_ids...
    squares.extend(ray.get(finished_ids))
duration = time.time() - start

In [11]:
assert squares == [0, 1, 4, 9]
assert duration < 4.1, f'duration = {duration}' 

## Exercise - "Homework" - in 02: Ray Actors Revisited

Since profiling shows that `live_neighbors` is the bottleneck, what could be done to reduce its execution time? The new implementation  solution shown here reduces its overhead by about 40%. Not bad. 

The solution also implements parallel invocations grid updates, rather doing the whole grid in sequential steps.

As discussed in lesson 4, these kinds of optimizations make sense when you _really_ have a compelling reason to squeeze optimal performance out of the code. Hence, this optimization exercise will mostly appeal to those of you with such requirements or who low-level performance optimizations like this. 

This solution for optimizing `live_neighbors` was developed using [micro-perf-tests.py](micro-perf-tests.py). The changes to the game code can be found in [game-of-life-2-exercise.py](game-of-life-2-exercise.py) rather than repeating them in cells here. Both scripts run standalone and both have a `--help` flag for more information.

If you tried the "easier experiments" suggested, such as enhancing `RayConwaysRules.step()` to accept a `num_steps` argument, you probably found that they didn't improve performance. As for the non-Ray game, this change only moves processing around but doesn't parallelize it more than before, so performance is about the same.

In [4]:
from game_of_life_2_exercise import RayGame, apply_rules_block, time_ray_games

For comparison, one set of test runs with the exercise code before improvements took about 25 seconds.

If you look at `RayGame2.step`, it calls `RayConwaysRules.step` one step at a time, using remote calls. This seems like a good place for improvement. Let's extend `RayConwaysRules.step` to do more than one step, just like `RayGame2.step` already supports.

Changes are indicated with comments.

In [14]:
time_ray_games(
    num_games = 1, 
    max_steps = 400, 
    batch_size = 1, 
    grid_dimensions = (100,100), 
    use_block_updates = False)

rules_ids:
[Actor(RayConwaysRules, b2e2e8931600)]
game_ids:
[Actor(RayGame, e717d8171600)]
Total time for 1 games (max_steps = 400, batch_size = 1) duration: 15.606 seconds


In a test run, this ran in about 15.5 seconds, about 9.5 seconds faster than the first version! Hence, as expected, optimizing `live_neighbors` provided significant improvement.

What about using block updates? Let's try a bigger grid, but fewer steps. First, without the block updates:

In [5]:
time_ray_games(
    num_games = 1, 
    max_steps = 100, 
    batch_size = 1, 
    grid_dimensions = (200,200), 
    use_block_updates = False)

rules_ids:
[Actor(RayConwaysRules, 5e95ba1b1800)]
game_ids:
[Actor(RayGame, 6a5ad72e1800)]
Total time for 1 games (max_steps = 100, batch_size = 1) duration: 15.319 seconds


In [6]:
time_ray_games(
    num_games = 1, 
    max_steps = 100, 
    batch_size = 1, 
    grid_dimensions = (200,200), 
    use_block_updates = True,
    block_size = 50)  # The default block size is -1, so no blocks are used!

rules_ids:
[Actor(RayConwaysRulesBlocks, f4f7ce371800)]
game_ids:
[Actor(RayGame, d2ae23331800)]
Total time for 1 games (max_steps = 100, batch_size = 1) duration:  8.000 seconds


In a test run, this performed about twice as fast! So block processing definitely helps.

Finally, does batching help? We'll use fewer steps and the original 100x100 grid. First without batching and then with batching:

In [7]:
%time time_ray_games(num_games = 1, max_steps = 100, batch_size = 1, grid_dimensions = (100,100), use_block_updates=False)

rules_ids:
[Actor(RayConwaysRules, f90511ea1800)]
game_ids:
[Actor(RayGame, 1520a0b51800)]
Total time for 1 games (max_steps = 100, batch_size = 1) duration:  3.901 seconds
CPU times: user 83.7 ms, sys: 35.4 ms, total: 119 ms
Wall time: 3.91 s


In [8]:
%time time_ray_games(num_games = 1, max_steps = 100, batch_size = 50, grid_dimensions = (100,100), use_block_updates=False)

rules_ids:
[Actor(RayConwaysRules, 13cf5f101800)]
game_ids:
[Actor(RayGame, 0d0441cf1800)]
Total time for 1 games (max_steps = 100, batch_size = 50) duration:  3.817 seconds
CPU times: user 59.7 ms, sys: 23.2 ms, total: 82.9 ms
Wall time: 3.82 s


Batching doesn't make much difference and in fact, we don't expect to matter, because it doesn't change the parallelism, like blocking does, and it doesn't make the algorithm more efficient, like the new `live_neighbors` does.

To conclude, the new implementation of `live_neighbors` has a noticable benefit. Batching doesn't make much difference, but using parallel blocks helps a lot.